In [1]:
import pandas as pd 
import numpy as np
import sys
import re
import string
from string import digits
import unidecode

import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from nltk.tokenize import word_tokenize
from nltk.tag.stanford import StanfordPOSTagger



data = pd.read_csv('../data/export_articles_EGC_2004_2018.csv', delimiter='\t')

In [2]:
#Si nettoyeur
root_path="/opt/StanfordTagguer/"
pos_tagger = StanfordPOSTagger(root_path + "models/french.tagger", root_path + "stanford-postagger.jar",encoding='utf8') #instance de la classe StanfordPOSTagger en UTF-8
def pos_tag(sentence):
    tokens = nltk.word_tokenize(sentence) #je transforme la phrase en tokens => si vous avez un texte avec plusieurs phrases, passez d'abord par nltk pour récupérer les phrases
    tags = pos_tagger.tag(tokens) #lance le tagging
    return tags

stemmer= FrenchStemmer()
stop_words = set(stopwords.words('perso'))
stop_words.add('a')
def clean_string(sentence):
     # All char to lower case ...
    cleaned_sentence = sentence.lower() 
    
    # Remove all digits
    remove_digit = str.maketrans('', '', digits)
    cleaned_sentence=cleaned_sentence.translate(remove_digit)
    # Remove all punctuation
    translator = re.compile('[%s]' % re.escape(string.punctuation))
    cleaned_sentence=translator.sub(' ', cleaned_sentence)
    # Remove all spaces
    cleaned_sentence=' '.join(cleaned_sentence.split())
    # Remove stopwords
    tokens = word_tokenize(cleaned_sentence)
    result = [i for i in tokens if not i in stop_words]
    cleaned_sentence = ' '.join(result)
    tokens = word_tokenize(cleaned_sentence)
    typePT= pos_tag(cleaned_sentence)
    result= []
    for item in typePT:
        if item[1]=='NC' or item[1]=='NP' or item[1]=='N' or item[1]=='NPP':
            result.append(item[0])
    cleaned_sentence = ' '.join(result)
    tokens = word_tokenize(cleaned_sentence)
    stemmed=[]
    
    for word in tokens:
        stemmed.append(stemmer.stem(word))
    cleaned_sentence = ' '.join(stemmed)
    return cleaned_sentence



In [3]:
data_abstract_title = pd.DataFrame(columns=['authors', 'title_abstract'])    

In [4]:
#Avec nettoyeur
for index, row in data.iterrows():
    print(round((index*100)/len(data), 2),"% / 100%", end='\r') # Print a loading bar ... 
    if str(row['abstract']) == "nan":
        data_abstract_title = data_abstract_title.append({'authors': row['authors'], 'title_abstract': clean_string(row['title'])}, ignore_index=True)
    else:
        data_abstract_title = data_abstract_title.append({'authors': row['authors'], 'title_abstract': clean_string(row['title'] + " "+ row['abstract'])}, ignore_index=True)
    
    #data_abstract_title = data_abstract_title.append({'num_doc': i, 'title_abstract': row['title'] + row['abstract']})

In [11]:
# Sans nettoyeur
for index, row in data.iterrows():
    print(round((index*100)/len(data), 2),"% / 100%", end='\r') # Print a loading bar ... 
    if str(row['abstract']) == "nan":
        data_abstract_title = data_abstract_title.append({'authors': row['authors'], 'title_abstract': row['title']}, ignore_index=True)
    else:
        data_abstract_title = data_abstract_title.append({'authors': row['authors'], 'title_abstract': row['title'] + " "+ row['abstract']}, ignore_index=True)
    
    #data_abstract_title = data_abstract_title.append({'num_doc': i, 'title_abstract': row['title'] + row['abstract']})

In [5]:
pd.set_option('display.max_rows', None)
#data_abstract_title

In [6]:
# Buid array with all authors names uniq
uniq_names = []
setUniq=set(uniq_names)
i = 0
for row in data_abstract_title.iterrows():
    athors_paper = data_abstract_title["authors"].iloc[i].split(', ')
    setAuthor=set(athors_paper)
    uniq_names = uniq_names + list(setAuthor-setUniq)
    setUniq=set(uniq_names)
    i=i+1
   

# Get number of authors
nb_authors = len(uniq_names)

nb_authors

2007

In [7]:
uniq_names

['Abdulhafiz Alkhouli',
 'Julien Longhi',
 'Claudia Marinica',
 'Nader Hassine',
 'Boris Borzic',
 'Fabrice Clérot',
 'Marc Boullé',
 'Marius Barctus',
 'Jean-Benoît Griesner',
 'Talel Abdesssalem',
 'Pierre Dosne',
 'Hubert Naacke',
 'Mohcine Maghfour',
 'Abdeljalil Elouardighi',
 'Fatima-Zahra Aazi',
 'Hafdalla Hammia',
 'Camille Pradel',
 'Elyase Lassouli',
 'Yasmine Mesbahi',
 'Damien Sileo',
 'Marwan Batrouni',
 'Aurélie Bertaux',
 'Christophe Nicolle',
 'Philippe Courtet',
 'Jorge López Castroman',
 'Romain Billot',
 'Philippe Lenca',
 'Mark Larsen',
 'Enrique Baca-García',
 'Michel Walter',
 'Sofian Berrouiguet',
 'Vincent Lemaire',
 'Oumaima Alaoui Ismaili',
 'Christophe Marsala',
 'Khalil Laghmari',
 'Mohammed Ramdani',
 'Emmanuelle Claeys',
 'Myriam Maumy-Bertrand',
 'Pierre Gançarski',
 'Fosca Giannotti',
 'Denis Maurel',
 'Sylvain Lefebvre',
 'Jérémie Sublime',
 'Jonathan Weber',
 'Michel Hassenforder',
 'Germain Forestier',
 'Bastien Latard',
 'Marie-Aude-Aufaure',
 'Étien

In [8]:
data_abstract_title_uniq = pd.DataFrame(columns=['author', 'title_abstract'])   

for name in uniq_names:
    data_abstract_title_uniq = data_abstract_title_uniq.append({'author': name, 'title_abstract': ""}, ignore_index=True)
    
    

In [9]:
data_abstract_title_uniq


,author,title_abstract
0,Abdulhafiz Alkhouli,
1,Julien Longhi,
2,Claudia Marinica,
3,Nader Hassine,
4,Boris Borzic,
5,Fabrice Clérot,
6,Marc Boullé,
7,Marius Barctus,
8,Jean-Benoît Griesner,
9,Talel Abdesssalem,


In [10]:
for i, rowi in data_abstract_title.iterrows():
    print(round((i*100)/len(data_abstract_title), 2),"% / 100%", end='\r') # Print a loading bar ... 
    athors_paper = data_abstract_title["authors"].iloc[i].split(', ')
    paper=data_abstract_title["title_abstract"].iloc[i]
    for author in athors_paper:
        data_abstract_title_uniq.loc[data_abstract_title_uniq["author"]==author,["title_abstract"]] = data_abstract_title_uniq.loc[data_abstract_title_uniq["author"]==author,["title_abstract"]]+ " " +paper

In [12]:
data_abstract_title_uniq

,title_abstract
author,
Abdulhafiz Alkhouli,citoyen éven object citoyen tweet éven cas él...
Julien Longhi,citoyen éven object citoyen tweet éven cas él...
Claudia Marinica,citoyen éven object citoyen tweet éven cas él...
Nader Hassine,citoyen éven object citoyen tweet éven cas él...
Boris Borzic,citoyen éven object citoyen tweet éven cas él...
Fabrice Clérot,clustering dat classif structuresous lign col...
Marc Boullé,clustering dat classif structuresous lign col...
Marius Barctus,clustering dat classif structuresous lign col...
Jean-Benoît Griesner,model clustering recommand recommand point in...


In [19]:
#Extraction sans nettoyeur
data_abstract_title_uniq.to_csv('../generated_data/AuthorAndTerms.csv', encoding="utf-8")

In [13]:
#Extraction avec nettoyeur
data_abstract_title_uniq.to_csv('../generated_data/AuthorAndTermsCleanedStemmed.csv', encoding="utf-8")

In [ ]:
data_abstract_title_uniq.iloc[400]["title_abstract"]

In [ ]:

uniq_words = []
for index, row in data_abstract_title_uniq.iterrows():
    tokens = word_tokenize(row["title_abstract"])
    for token in tokens:
        if token not in uniq_words:
            uniq_words.append(token)

In [ ]:
print(len(uniq_words))

In [ ]:
zero_matrix = np.zeros((len(uniq_names), len(uniq_words)))

In [ ]:
# Create publication adjacency matrix 
names = [_ for _ in uniq_names]
term_freq_matrix = pd.DataFrame(zero_matrix, index=uniq_names, columns=uniq_words)

In [ ]:
term_freq_matrix

In [ ]:
def author_term(author,sentence, adjecency_matrix):
    allWords=word_tokenize(sentence)
    for word in allWords:
        adjecency_matrix[word][author] = adjecency_matrix[word][author] + 1
       


In [ ]:
i=0
for row in data_abstract_title_uniq.iterrows():
    author=data_abstract_title_uniq["author"].iloc[i]
    abstract=data_abstract_title_uniq["title_abstract"].iloc[i]
    author_term(author,abstract,term_freq_matrix)
    i=i+1


In [ ]:
term_freq_matrix

In [ ]:
term_freq_matrix.to_csv('../generated_data/outputTerms.csv', encoding="utf-8")

In [7]:

data_freq = pd.read_csv('../generated_data/outputTerms.csv',index_col=0, delimiter=',')

In [8]:
data_freq

,citoyenne,événements,objectif,citoyens,tweets,cas,élection,messages,candidats,caractéristiques,...,ranking,creations,combination,scheme,conducted,duc,lowest,panels,photos,granulaire
Abdulhafiz Alkhouli,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Nader Hassine,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Claudia Marinica,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Julien Longhi,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Boris Borzic,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Marius Barctus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fabrice Clérot,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Marc Boullé,0.0,0.0,1.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jean-Benoît Griesner,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Talel Abdesssalem,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
#récupération
data_abstract_title_uniq = pd.read_csv('../generated_data/AuthorAndTermsCleanedStemmed.csv', delimiter=',')

In [18]:
data_abstract_title_uniq

,author,title_abstract
0,Abdulhafiz Alkhouli,citoyen éven object citoyen tweet éven cas él...
1,Julien Longhi,citoyen éven object citoyen tweet éven cas él...
2,Claudia Marinica,citoyen éven object citoyen tweet éven cas él...
3,Nader Hassine,citoyen éven object citoyen tweet éven cas él...
4,Boris Borzic,citoyen éven object citoyen tweet éven cas él...
5,Fabrice Clérot,clustering dat classif structuresous lign col...
6,Marc Boullé,clustering dat classif structuresous lign col...
7,Marius Barctus,clustering dat classif structuresous lign col...
8,Jean-Benoît Griesner,model clustering recommand recommand point in...
9,Talel Abdesssalem,model clustering recommand recommand point in...


In [19]:
data_abstract_title_uniq.to_csv('../generated_data/AuthorAndTermsCleanedStemmed.csv', encoding="utf-8")

In [20]:
data_abstract_title_uniq

,author,title_abstract
0,Abdulhafiz Alkhouli,citoyen éven object citoyen tweet éven cas él...
1,Julien Longhi,citoyen éven object citoyen tweet éven cas él...
2,Claudia Marinica,citoyen éven object citoyen tweet éven cas él...
3,Nader Hassine,citoyen éven object citoyen tweet éven cas él...
4,Boris Borzic,citoyen éven object citoyen tweet éven cas él...
5,Fabrice Clérot,clustering dat classif structuresous lign col...
6,Marc Boullé,clustering dat classif structuresous lign col...
7,Marius Barctus,clustering dat classif structuresous lign col...
8,Jean-Benoît Griesner,model clustering recommand recommand point in...
9,Talel Abdesssalem,model clustering recommand recommand point in...
